In [80]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import roc_curve, auc
from torch.utils.data import random_split, DataLoader


from customDatasets.audioDataset import AudioDataset


In [81]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )

        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_size),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [82]:
def train_model(model, train_dl, val_dl, criterion, optimizer, device, epochs=5):
    train_losses = []
    val_losses = []
    for epoch in range(epochs):
        train_losses = []
        val_losses = []

        for inputs, labels in train_dl:
            model.train()
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, inputs.view(inputs.size(0), -1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())

        print(f'Epoch[{epoch + 1}/{epochs}], Loss: {np.average(train_losses): .4f}')
        
        # for inputs, labels in val_dl:
        #     model.eval()
        #     with torch.no_grad():
        #         inputs, labels = inputs.to(device), labels.to(device)
        #         outputs = model(inputs)
        #         loss = criterion(outputs, inputs)
        #         val_losses.append(loss.item())
        
        scores = []
        full_labels = []
        for inputs, labels in val_dl:
            model.eval()
            with torch.no_grad():
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                mse = torch.sum((outputs - inputs.view(inputs.size(0), -1)) ** 2, dim=1) / outputs.shape[1]
                scores.append(mse)
                full_labels.append(labels)
        
        full_labels = torch.cat([label for label in full_labels])
        scores = torch.cat([score for score in scores])
        fpr, tpr, _ = roc_curve(full_labels.cpu().detach(), scores.cpu().detach(), pos_label=0)
        roc_auc = auc(fpr, tpr)
        print(roc_auc)

      

        

            
        # print(f'Epoch[{epoch + 1}/{epochs}], Val Loss: {np.average(val_losses): .4f}')



In [83]:
def set_seed(seed = 42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

CONFIG = {
    "seed": 42,
    "epochs": 20,
    "num_classes": 2,
    "learning_rate": 0.001,
    "train_batch_size": 32,
    "val_batch_size": 16,
    "test_batch_size": 128,
    "criterion": nn.MSELoss(),
    "device":
        torch.device(
            "cuda:0" if torch.cuda.is_available()
            else "mps" if torch.backends.mps.is_available()
            else "cpu"
        )
}

set_seed(CONFIG['seed'])

data_path = "./data/train/"
data_path_test = "./data/test/"


meta_train_df = pd.read_csv("./data/train.csv")
meta_test_df = pd.read_csv("./data/test.csv")

train_df = meta_train_df[['filename', 'is_normal', 'machine_id']]
train_ds = AudioDataset(train_df, data_path)
test_df = meta_test_df[['filename', 'is_normal', 'machine_id']]
test_ds = AudioDataset(test_df, data_path_test)

num_items = len(test_ds)
num_test = round(num_items * 0.3)
num_val = num_items - num_test

test_ds, val_ds = random_split(test_ds, [num_test, num_val])


train_dl = DataLoader(train_ds, batch_size=CONFIG['train_batch_size'], shuffle=True)
val_dl = DataLoader(val_ds, batch_size=CONFIG['val_batch_size'], shuffle=False)
test_dl = DataLoader(test_ds, batch_size=CONFIG["test_batch_size"], shuffle=False)

In [84]:
input_size = 64 * 313 # TODO: move to config
model = Autoencoder(input_size, encoding_dim=8)
model = model.to(CONFIG["device"])
optimizer = optim.Adam(model.parameters(), lr=CONFIG["learning_rate"])

In [85]:
train_model(model, train_dl, val_dl, CONFIG["criterion"], optimizer, CONFIG["device"], CONFIG["epochs"])

Epoch[1/20], Loss:  145.5195
0.19820582261340552
Epoch[2/20], Loss:  130.3770
0.20298747461069738
Epoch[3/20], Loss:  108.1194
0.2018872714962762
Epoch[4/20], Loss:  80.2549
0.2182549085985105
Epoch[5/20], Loss:  55.5054
0.24681787406905886
Epoch[6/20], Loss:  38.8373
0.37514387271496275
Epoch[7/20], Loss:  30.1414
0.5363236289776574
Epoch[8/20], Loss:  26.4707
0.6453283683141503
Epoch[9/20], Loss:  24.9881
0.6902970548408938
Epoch[10/20], Loss:  24.3710
0.7071174678402166
Epoch[11/20], Loss:  24.3831
0.7129823967501693
Epoch[12/20], Loss:  24.1252
0.7172393364928911
Epoch[13/20], Loss:  24.0133
0.7120260663507109
Epoch[14/20], Loss:  23.8831
0.7212593094109682
Epoch[15/20], Loss:  23.9081
0.709580230196344
Epoch[16/20], Loss:  23.9378
0.7138794854434665
Epoch[17/20], Loss:  24.1835
0.7186780636425186
Epoch[18/20], Loss:  23.8568
0.7217163168584969
Epoch[19/20], Loss:  24.3755
0.7178402166553826
Epoch[20/20], Loss:  23.9587
0.7192281651997292
